In [3]:
import pandas as pd
from tkinter import *
data_df = pd.read_pickle("data/data.pkl")
data_c1 = data_df[(data_df['Cluster'] == 1)]
data_c2 = data_df[(data_df['Cluster'] == 2)]
data_c3 = data_df[(data_df['Cluster'] == 3)]

In [ ]:
import requests
from bs4 import BeautifulSoup
import string
data_df['Genre_match']=0
data_df['Volume_match']=0
data_df['Author_match']=0
data_df['Cluster_match']=0
def raise_frame(frame):
    frame.tkraise()
    

root = Tk()
root.iconbitmap('assets/favicon.ico')

root.title('Recommendation Tool')

f1 = Frame(root)
f2 = Frame(root)
for frame in (f1, f2):
    frame.grid(row=0, column=0)
root.geometry("+470+100")


f1.configure(background='white')
f2.configure(background='SteelBlue2')


def submit(frame):
    raise_frame(frame)
    for checkbox in genre_checkboxes:
        if checkbox.get()!="":
            data_df.loc[data_df.Genre == checkbox.get().capitalize(), 'Genre_match'] = 1
    for checkbox in volume_checkboxes:
        if checkbox.get()!="":
            data_df.loc[data_df.Volume == checkbox.get(), 'Volume_match'] = 1
    for index,checkbox in enumerate (cluster_checkboxes):
        if checkbox.get()!=0:
            data_df.loc[data_df.Cluster == checkbox.get(), 'Cluster_match'] = 1
            data_df.loc[data_df.Author == data_df.Author[samples[index]], 'Author_match'] = 1
            
    global book_name
    
    if selection==0:
        
        book=""
        for index, value in enumerate(data_df.index): 
            bn=book_name.get().replace('(','\(')
            bn=bn.replace(')','\)')
            test=re.search(bn, value,re.IGNORECASE)
            if test!= None:
                book= test[0]
        #print(book)
        if(len(book)>0):
            cluster=data_df.loc[data_df.index==book]['Cluster'].values[0]
            author=data_df.loc[data_df.index == book]['Author'].values[0]
            genre=data_df.loc[data_df.index == book]['Genre'].values[0]


            #global author
            #set attribute scores based on the user's input
            data_df.loc[data_df.Cluster == cluster, 'Cluster_match'] = 1
            data_df.loc[data_df.Author == author,'Author_match'] = 1
            data_df.loc[data_df.Genre == genre,'Genre_match'] = 1
        
    #open goodreads url
    import webbrowser
    def callback(url):
        webbrowser.open_new(url)
    
    recommend_df = data_df[['Rating','NB_Ratings','Genre_match','Volume_match','Cluster_match','Author_match']]
    normalized_recommend=normalize(recommend_df)
    column_list = list(normalized_recommend)[2:]
    ratings_list=['Rating','NB_Ratings']
    
    #Calculate preference score
    normalized_recommend['Score']=normalized_recommend[ratings_list].sum(axis=1)/2+normalized_recommend[column_list].sum(axis=1)
    normalized_recommend.sort_values(by=['Score'])
    normalized_recommend.head()
    #print(normalized_recommend.sort_values(by=['Score'],ascending=False))
    f2.update()
    
    #Recommendation window items
    b0=Label(f2, text='The engine recommends these books:',font=('Raleway', 18,'bold'),fg='gray15',background='SteelBlue2')
    b0.grid(row=0,columnspan=3,ipady=20,ipadx=50)
    b1 = Label(f2, text=str(normalized_recommend.sort_values(by=['Score'],ascending=False).index.values[0]),font=('Open Sans', 12),cursor="hand1",borderwidth=1,width=100,justify="center", relief="solid",background='white')
    b1.grid(row=1,columnspan=3,ipadx=100,ipady=20)
    
    b2 = Label(f2, text=str(normalized_recommend.sort_values(by=['Score'],ascending=False).index.values[1]),font=('Open Sans', 12),cursor="hand1",borderwidth=1,width=100,justify="center", relief="solid",background='white')
    b2.grid(row=2,columnspan=3,ipadx=100,ipady=20)

    b3 = Label(f2, text=str(normalized_recommend.sort_values(by=['Score'],ascending=False).index.values[2]),font=('Open Sans', 12),cursor="hand1",borderwidth=1,width=100,justify="center", relief="solid",background='white')
    b3.grid(row=3,columnspan=3,ipadx=100,ipady=20)

    b4 = Label(f2, text=str(normalized_recommend.sort_values(by=['Score'],ascending=False).index.values[3]),font=('Open Sans', 12),cursor="hand1",borderwidth=1,width=100,justify="center", relief="solid",background='white')
    b4.grid(row=4,columnspan=3,ipadx=100,ipady=20)

    b5 = Label(f2, text=str(normalized_recommend.sort_values(by=['Score'],ascending=False).index.values[4]),font=('Open Sans', 12),cursor="hand1",borderwidth=1,width=100, justify="center", relief="solid",background='white')
    b5.grid(row=5,columnspan=3,ipadx=100,ipady=20)
    labels=[b1,b2,b3,b4,b5]
    
    bookurls=[]
    for label in labels:
        baseUrl = 'https://www.goodreads.com/search?q='
        path = label['text'].split('(')[0]
        data = requests.get(baseUrl+path).text
        soup = BeautifulSoup(data, "html.parser")
        link=soup.find('a',attrs={'class':"bookTitle",'itemprop':'url'})

        bookurls.append("https://www.goodreads.com"+link['href'])
    b1.bind("<Button-1>", lambda e: callback(bookurls[0]))
    b2.bind("<Button-1>", lambda e: callback(bookurls[1]))
    b3.bind("<Button-1>", lambda e: callback(bookurls[2]))
    b4.bind("<Button-1>", lambda e: callback(bookurls[3]))
    b5.bind("<Button-1>", lambda e: callback(bookurls[4]))

    normalized_recommend.index.values[:6]
    global f1
    f1.destroy()
    root.geometry("+200+200")
    raise_frame(f2)
    
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

def toggle(var,widget):
    if var.get()!="":
        widget.grid(row=4,column=1,ipady=5)
    else:
        widget.grid_remove()
    

fiction = StringVar()
nonfiction = StringVar()
fantasy = StringVar()
ya = StringVar()
history = StringVar()
mystery = StringVar()
classics = StringVar()
horror = StringVar()
romance = StringVar()
other = StringVar()
e= StringVar()
photoimage=PhotoImage(file="assets/button.png")

#1st FRAME
Label(f1, text="What genres are you most interested in?", wraplength=500,font=('Raleway', 16,'bold'),background='white').grid(row=0,columnspan=3,padx=(10,0),pady=20)
genre_checkboxes=[fiction,nonfiction,fantasy,ya,history,mystery,classics,horror,romance,e]
Checkbutton(f1, text="Fiction", variable=fiction,onvalue='Fiction',offvalue="",background='white',bd=10, anchor="w",width=10,font=('Open Sans', 12)).grid(row=1,column=0)
Checkbutton(f1, text="Nonfiction", variable=nonfiction,onvalue='Nonfiction',offvalue="",background='white',anchor="w",width=10,bd=10,font=('Open Sans', 12)).grid(row=2)
Checkbutton(f1, text="Fantasy", variable=fantasy,onvalue='Fantasy',offvalue="",background='white',bd=10,anchor="w",width=10,font=('Open Sans', 12)).grid(row=3)
Checkbutton(f1, text="Young Adult", variable=ya,onvalue='Young Adult',offvalue="",background='white',width=10,anchor="w",bd=10,font=('Open Sans', 12)).grid(row=1,column=1)
Checkbutton(f1, text="Historical", variable=history,onvalue='Historical',offvalue="",background='white',width=10,anchor="w",bd=10,font=('Open Sans', 12)).grid(row=2,column=1 )
Checkbutton(f1, text="Mystery", variable=mystery,onvalue='Mystery',offvalue="",background='white',width=10,bd=10,anchor="w",font=('Open Sans', 12)).grid(row=3,column=1)
Checkbutton(f1, text="Classics", variable=classics,onvalue='Classics',offvalue="",background='white',width=10,anchor="w",bd=10,font=('Open Sans', 12)).grid(row=1,column=2, sticky=E)
Checkbutton(f1, text="Horror", variable=horror,onvalue='Horror',offvalue="",background='white',width=10,bd=10,anchor="w",font=('Open Sans', 12)).grid(row=2,column=2, sticky=E)
Checkbutton(f1, text="Romance", variable=romance,onvalue='Romance',offvalue="",background='white',width=10,anchor="w",bd=10,font=('Open Sans', 12)).grid(row=3,column=2, sticky=E)
entry=Entry(f1,textvariable=e,width=45,borderwidth=1)
ck=Checkbutton(f1, text="Other", variable=other, command=lambda: toggle(other,entry),offvalue="",onvalue="Other",anchor="w",width=10,background='white',bd=10,font=('Open Sans', 12))
ck.grid(row=4)

#2nd FRAME
short = StringVar()
medium = StringVar()
long = StringVar()
volume_checkboxes=[short,medium,long]
Label(f1, text="What book length do you prefer?",font=('Raleway', 16,'bold'),background='white').grid(row=5,columnspan=3,padx=(10,0),pady=20)
Checkbutton(f1, text="Short", variable=short,onvalue='Short',offvalue="",background='white',anchor="w",width=10,font=('Open Sans', 12)).grid(row=6,column=0)
Checkbutton(f1, text="Medium", variable=medium,onvalue='Medium',offvalue="",background='white',anchor="w",width=10,font=('Open Sans', 12)).grid(row=6,column=1)
Checkbutton(f1, text="Long", variable=long,onvalue='Long',offvalue="",background='white',anchor="w",width=10,font=('Open Sans', 12)).grid(row=6,column=2)

#3rd FRAME
sel=Label(f1, text="Select recommendation type",font=('Raleway', 16,'bold'),background='white')
sel.grid(row=7,columnspan=3,pady=20)
input_b=Button(f1, text = 'Input-based recommendation',border=1,font=('Source Code Pro', 14),background='snow',width = 30,command= lambda:input_rc())
input_b.grid(row=8,columnspan=3)
select_b=Button(f1, text = 'Selection-based recommendation',border=1,font=('Source Code Pro', 14),background='snow',width = 30,command= lambda:select_rc())
select_b.grid(row=9,columnspan=3)
book_name=Entry(f1,font = "Roboto 18",fg='gray50',justify='center',width=45,borderwidth=1)
selection=0
def input_rc():
    
    global f1,sel,book_name,selection
    global input_b
    global select_b
    selection=0
    sel.grid_forget()
    input_b.grid_forget()
    select_b.grid_forget()
    Label(f1, text="Enter a book name:",font=('Raleway', 16,'bold'),background='white').grid(row=7,column=1,pady=20)
    book_name.grid(row=9,padx=10,ipady=10,columnspan=3)
    

lb3=Label(f1, text="Which of these books do you prefer?",font=('Raleway', 16,'bold'),background='white')
c1 = IntVar()
c2 = IntVar()
c3 = IntVar()
cluster_checkboxes=[c1,c2,c3]

samples=[data_c1.sample().index[0],data_c2.sample().index[0],data_c3.sample().index[0]]
cb1=Checkbutton(f1, text=samples[0], variable=c1,onvalue=1,offvalue=0,background='white',bd=10,font=('Open Sans', 12))
cb2=Checkbutton(f1, text=samples[1], variable=c2,onvalue=2,offvalue=0,background='white',bd=10,font=('Open Sans', 12))
cb3=Checkbutton(f1, text=samples[2], variable=c3,onvalue=3,offvalue=0,background='white',bd=10,font=('Open Sans', 12))
    
def select_rc():
    global f1,cb1,cb2,cb3,lb3,select_b,input_b,rf_b,selection
    selection=1
    input_b.grid_forget()
    select_b.grid_forget()
    lb3.grid(row=7,column=1,pady=20)
    cb1.grid(row=8,columnspan=3, sticky=W)
    cb2.grid(row=9,columnspan=3, sticky=W)
    cb3.grid(row=10,columnspan=3, sticky=W)
    rf_b.grid(row=11,column=0,columnspan=3)
  


def refresh():
    global samples
    samples=[data_c1.sample().index[0],data_c2.sample().index[0],data_c3.sample().index[0]]
    cb1.config(text=samples[0])
    cb2.config(text=samples[1])
    cb3.config(text=samples[2])
#Button(master, text='Quit', command=master.quit).grid(row=12, sticky=W, pady=4)
photo = PhotoImage(file = "assets/refresh.png")
rf_b=Button(f1, text = 'Refresh books', image = photo,border=0,command= lambda:refresh())
Button(f1, text='Run Recommendation', command= lambda: submit(f2),border="1",width=30,font=('Roboto', 15,'bold'),fg='white',bg='IndianRed1').grid(row=12, columnspan=3, pady=4)

raise_frame(f1)

root.mainloop()

In [2]:
data_df

,Author,Genre,Description,Rating,NB_Ratings,NB_Pages,Format,Volume,Cluster
Title,,,,,,,,,
The Girl Who Played with Fire (Millennium #2),Stieg Larsson,Fiction,"\nPart blistering espionage thriller, part riv...",4.24,788081,503,Hardcover,Medium,3.0
The Da Vinci Code (Robert Langdon #2),Dan Brown,Fiction,\nAn ingenious code hidden in the works of Leo...,3.85,1867702,489,Paperback,Medium,3.0
The Help,Kathryn Stockett,Fiction,\nThree ordinary women are about to take one e...,4.47,2081569,451,Hardcover,Medium,3.0
The Girl with the Dragon Tattoo (Millennium #1),Stieg Larsson,Fiction,"\nHarriet Vanger, a scion of one of Sweden’s w...",4.13,2517604,465,Hardcover,Medium,1.0
Harry Potter and the Sorcerer's Stone (Harry Potter #1),J.K. Rowling,Fantasy,\nAlternate cover edition of ISBN 978043955493...,4.47,6714911,309,Hardcover,Medium,3.0
...,...,...,...,...,...,...,...,...,...
Quit or Die Trying,Alison Beard,Nonfiction,\nWhen smokers coughed and coughed and couldn'...,4.35,20,256,Kindle Edition,Short,2.0
"A Year of Living Prayerfully: How a Curious Traveler Met the Pope, Walked on Coals, Danced with Rabbis, and Revived His Prayer Life",Jared Brock,Nonfiction,\nIs Anyone Listening? Do My Prayers Really Ch...,4.18,499,352,Hardcover,Medium,2.0
Slow Down,Lee Matthew Goldberg,Fiction,\nHow far would you go to make your dreams com...,3.97,32,270,Paperback,Short,3.0


In [2]:
data_df.head()
    

,Author,Genre,Description,Rating,NB_Ratings,NB_Pages,Format,Volume,Cluster,Series
Title,,,,,,,,,,
The Girl Who Played with Fire (Millennium #2),Stieg Larsson,Fiction,"\nPart blistering espionage thriller, part riv...",4.24,788081,503,Hardcover,Medium,1,1
The Da Vinci Code (Robert Langdon #2),Dan Brown,Fiction,\nAn ingenious code hidden in the works of Leo...,3.85,1867702,489,Paperback,Medium,2,1
The Help,Kathryn Stockett,Fiction,\nThree ordinary women are about to take one e...,4.47,2081569,451,Hardcover,Medium,1,0
The Girl with the Dragon Tattoo (Millennium #1),Stieg Larsson,Fiction,"\nHarriet Vanger, a scion of one of Sweden’s w...",4.13,2517604,465,Hardcover,Medium,3,1
Harry Potter and the Sorcerer's Stone (Harry Potter #1),J.K. Rowling,Fantasy,\nAlternate cover edition of ISBN 978043955493...,4.47,6714911,309,Hardcover,Medium,2,1
